<a href="https://colab.research.google.com/github/13727054/Ai_project_recommandation_system_of_books/blob/main/AI_4300SEF_project_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install --upgrade pip setuptools wheel
!pip install implicit


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.6 MB/s  0:00:00m eta 0:00:01
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 72.1.0
    Uninstalling setuptools-72.1.0:
      Successfully uninstalled setuptools-72.1.0 0/2 [setuptools]
  Attempting uninstall: pip━━━━━━━━━━━━━━━━━━━━━ 0/2 [setuptools]
    Found existing installation: pip 25.2━━━ 0/2 [setuptools]
    Uninstalling pip-25.2:━━━━━━━━━━━━━━━━━━ 0/2 [setuptools]
      Successfully uninstalled pip-25.2━━━━━ 0/2 [setuptools]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pip]1/2 [pip]
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp313-cp313-macosx_15_0_arm64.whl size=675065 sha256=a3e8e4edbe4d54b5c73940142aa5819fc431de95b316

In [2]:
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import implicit
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
# Step 1: Download datasets
#!wget https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv
#!wget https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv

# Step 2: Load datasets
books = pd.read_csv("books.csv")
ratings = pd.read_csv("ratings.csv")

In [5]:
# Step 3: Content-based similarity (TF-IDF on titles)
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(books["title"].fillna(""))
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def content_recommend(book_title, top_n=5):
    idx = books[books["title"] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    book_indices = [i[0] for i in sim_scores]
    return books.iloc[book_indices][["book_id", "title"]]

In [6]:
# Step 4: Collaborative filtering with implicit ALS
# Build user-item matrix
user_item = sp.coo_matrix(
    (ratings["rating"], (ratings["user_id"], ratings["book_id"]))
).tocsr()

# Train ALS model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
model.fit(user_item)

def collaborative_recommend(user_id, top_n=5):
    recommended = model.recommend(user_id, user_item[user_id], N=top_n)
    # recommended is a list of (book_id, score) or similar
    book_ids = [r[0] for r in recommended]   # take the book_id
    return books[books["book_id"].isin(book_ids)][["book_id", "title"]]

/opt/anaconda3/lib/python3.13/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [20]:
print(collaborative_recommend(user_id=2))

    book_id                                              title
91       92  Freakonomics: A Rogue Economist Explores the H...


In [7]:
# Step 5: Hybrid recommendation
def hybrid_recommend_mmr(user_id, book_title, top_n=5, alpha=0.5, lambda_div=0.7, disliked_books=None):
    if disliked_books is None:
        disliked_books = []

    # Content-based scores
    idx = books[books["title"] == book_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Collaborative predictions
    recommended = model.recommend(user_id, user_item[user_id], N=200)
    collab_preds = {r[0]: r[1] for r in recommended}

    # Combine scores
    candidates = []
    for i, score in sim_scores:
        book_id = books.iloc[i]["book_id"]
        if book_id in disliked_books:
            continue
        collab_score = collab_preds.get(book_id, 0)
        final_score = alpha * collab_score + (1 - alpha) * score
        candidates.append((book_id, final_score))

    candidate_df = pd.DataFrame(candidates, columns=["book_id", "score"])
    candidate_df = candidate_df.merge(books[["book_id", "title"]], on="book_id")

    # Precompute similarity matrix for diversity
    tfidf_matrix = tfidf.transform(candidate_df["title"].fillna(""))
    sim_matrix = cosine_similarity(tfidf_matrix)

    # MMR re-ranking
    selected_books = []
    while len(selected_books) < top_n and len(candidate_df) > 0:
        if not selected_books:
            # pick most relevant first
            idx_best = candidate_df["score"].idxmax()
        else:
            mmr_scores = []
            for i in candidate_df.index:
                relevance = candidate_df.loc[i, "score"]
                # diversity: max similarity to already selected books
                selected_idx = [candidate_df.index.get_loc(j) for j in selected_books]
                diversity = max(sim_matrix[i, selected_idx]) if selected_idx else 0
                mmr_score = lambda_div * relevance - (1 - lambda_div) * diversity
                mmr_scores.append((i, mmr_score))
            idx_best = max(mmr_scores, key=lambda x: x[1])[0]

        # ✅ FIX: store book_id instead of row index
        selected_books.append(candidate_df.loc[idx_best, "book_id"])
        candidate_df = candidate_df.drop(idx_best)

    # Return the selected book_ids
    return books[books["book_id"].isin(selected_books)][["book_id", "title"]]

In [47]:
disliked = [7, 466]

print("Hybrid + MMR recommendations for user 1 based on 'The Hobbit':")
#uesrid= input id(retrived from database)
print(hybrid_recommend_mmr(user_id=3, book_title="The Hobbit", top_n=5, alpha=0.5, lambda_div=0.7))

print("\nUpdated recommendations after dislikes:")
print(hybrid_recommend_mmr(user_id=4, book_title="The Hobbit", top_n=5, alpha=0.5, lambda_div=0.7, disliked_books=disliked))

Hybrid + MMR recommendations for user 1 based on 'The Hobbit':
      book_id                                              title
6           7                                         The Hobbit
13         14                                        Animal Farm
465       466                          The Hobbit: Graphic Novel
963       964  J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
1128     1129   The History of the Hobbit, Part One: Mr. Baggins

Updated recommendations after dislikes:
      book_id                                              title
0           1            The Hunger Games (The Hunger Games, #1)
1           2  Harry Potter and the Sorcerer's Stone (Harry P...
58         59                                    Charlotte's Web
963       964  J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...
1128     1129   The History of the Hobbit, Part One: Mr. Baggins
